# Data

[150k Python Dataset]("https://eth-sri.github.io/py150") from SRILAB.

# Code representation

A continuous vector embedding of each code fragment at method–level granularity as "document".[13] FastText, a variation of Word2Vec algorithm.
- Extracting Information from Source Code
    - simple tokenizer: extract all words from source code by removing non–alphanumeric tokens.=> indifferenciable
    - parser-based approach: traverse through the parse tree for each method, and extract information from the following syntactic categories. (Java-like)
        - method name
        - method invocation
        - Enums
        - String literals
        - comments
        - <strike>variable name</strike>
- Building Vector Representations
    - <strike>simply average embeddings</strike>
    - Weighted average of all unique words in a document=> normalized tf-idf
- Retrieval
    - average the vector representations of constituent words to create a document embedding for the query sentence
    - a standard similarity search algorithm to find the document vectors with closest cosine distance. => FAISS 

# Model

Input: natural language queries <br>
Output: related code fragments retrieved directly from Github code corpus<br><br>

Map the Input into the same vector space as the codebase, and then calculate the vector distance of them in order to get the relevant result.

# Evaluation

Metric and choosing parameters of the model

- Metric: select subsets of words from the document as simulated queries and then see if it can retrive the document, and then evaluate by the percentage of the documents that are retrieve back at top1 and top10. 
    - random benchmark test
    - TF-IDF benchmark test => better performance
- Parameters:
    - embedding dimention=> 500
    - three ways of combining word embeddings to document embeddings=> the conclusion is tf-idf better
    - vector representation=> BM25 is better

# Implementation

In [1]:
import numpy as np
import pandas as pd

from gensim.models import FastText

## Data Processing

In [2]:
# load in processed file. A set of keywords for each document (source code function)
unpickled_df = pd.read_pickle("data/py100.pkl")
func_size=len(unpickled_df)
print("Total Number of Functions: {}".format(func_size))
unpickled_df.head()

Total Number of Functions: 1038


,data_id,function_name,docstring,func_call
0,1,__getattribute__,,__getattribute__
1,1,__setattr__,,"ref,__setattr__"
2,2,main,,"setup,closing,ZmqProxy,consume_in_thread,wait"
3,3,test_vpnservice_create,,"create_stubs,first,AndReturn,ReplayAll,vpnserv..."
4,3,test_vpnservices_get,,"create_stubs,AndReturn,ReplayAll,vpnservices_g..."


In [3]:
# for each function, combine all the keywords into a set
list_function_keywords=[]
for idx in range(len(unpickled_df)):
    keywords=[]
    keywords.append(unpickled_df.iloc[idx]["function_name"].lower())
    
    #[TODO] only alphabenumeric characters
    #[TODO] camel case
    #[TODO] snake case
    #[TODO] Add function calls
    docstring=unpickled_df.iloc[idx]["docstring"].lower().split()
        
    func_invoc=unpickled_df.iloc[idx]["func_call"].lower().split(",")
    
    keywords+=docstring
    keywords+=func_invoc
    list_function_keywords.append(set(keywords))

In [4]:
list_function_keywords

[{'__getattribute__'},
 {'__setattr__', 'ref'},
 {'closing', 'consume_in_thread', 'main', 'setup', 'wait', 'zmqproxy'},
 {'andreturn',
  'assertisinstance',
  'create_stubs',
  'create_vpnservice',
  'first',
  'replayall',
  'request',
  'test_vpnservice_create',
  'vpnservice',
  'vpnservice_create'},
 {'andreturn',
  'assertisinstance',
  'asserttrue',
  'create_stubs',
  'id',
  'list',
  'list_vpnservices',
  'name',
  'replayall',
  'request',
  'test_vpnservices_get',
  'vpnservice',
  'vpnservices_get'},
 {'andreturn',
  'assertisinstance',
  'create_stubs',
  'first',
  'replayall',
  'request',
  'show_vpnservice',
  'test_vpnservice_get',
  'vpnservice',
  'vpnservice_get'},
 {'andreturn',
  'assertisinstance',
  'create_ikepolicy',
  'create_stubs',
  'first',
  'ikepolicy',
  'ikepolicy_create',
  'replayall',
  'request',
  'test_ikepolicy_create'},
 {'andreturn',
  'assertisinstance',
  'asserttrue',
  'create_stubs',
  'id',
  'ikepolicies_get',
  'ikepolicy',
  'list',

## Building Word Embeddings

In [15]:
# hyperparameters
vocab_size=100
window_size=5
min_count=1


# other parameters defined earlier
# func_size

In [16]:
# We employ the continuous skip–gram model with a window size of 5, 
# i.e. all pairs of words within distance 5 are considered nearby words.

#[TODO] tuning hyperparameters
model = FastText(size=vocab_size, window=window_size, min_count=min_count)  # instantiate
model.build_vocab(sentences=list_function_keywords)
model.train(sentences=list_function_keywords, total_examples=len(list_function_keywords), epochs=10)  # train

In [17]:
print(model)

FastText(vocab=2918, size=100, alpha=0.025)


In [18]:
# saving a model trained via Gensim's fastText implementation
model.save('saved_model_gensim')

In [19]:
trained_ft_vectors = model.wv
# save vectors to file if you want to use them later
trained_ft_vectors.save_word2vec_format('embeddings.txt', binary=False)

In [25]:
# Test
trained_ft_vectors.most_similar("click", topn=10)

[('uuid', 0.9988144636154175),
 ('autogen_tools', 0.9987874031066895),
 ('_lookup_xml_from_soup', 0.9987666606903076),
 ('`src_path`:', 0.9987651109695435),
 ('drive', 0.9987644553184509),
 ('scenetimewarp', 0.998760461807251),
 ('lookup', 0.998759388923645),
 ('clickedsignal', 0.9987587332725525),
 ('(delete)', 0.9987566471099854),
 ('tweets_from_tiids', 0.9987522959709167)]

## Building Document Embeddings

1. Average over all the words;
2. Average over the unique words in each document;
3. [x] Weighted average of all unique words in a document

In [21]:
trained_ft_vectors["ping"]

array([-0.30873144,  0.04389549,  0.36451772,  0.20677614, -0.42488563,
       -0.00816474, -0.00272692,  0.17229767, -0.26310018,  0.8255236 ,
       -0.5982063 ,  0.3263564 , -0.34125307, -0.7620928 ,  0.30430394,
        0.20494114, -0.0985655 , -0.08308541,  0.04502838,  0.56351703,
       -0.56962126,  0.29001042,  0.42345685,  0.24321483,  0.42053938,
       -0.8196062 ,  0.35603952,  0.47772178, -0.4872069 , -0.11585229,
        0.5361255 , -0.02294187, -0.3006934 , -0.37413743,  0.29466164,
        0.20686015, -0.2751746 , -0.51173466,  0.04152834,  0.8488972 ,
        0.24582677,  0.12246843, -0.05783413,  0.33660105,  0.46147385,
        0.29725516, -0.4300252 ,  0.17061587, -0.259805  ,  0.23556824,
        0.22309585,  0.11666885, -0.08638151, -0.18883961, -0.13002938,
        0.00468559,  0.28211114, -0.39217386, -0.11985342,  0.14492673,
        0.03596745, -0.12084921, -0.16670412, -0.71959513,  0.5438955 ,
       -0.09593102, -0.03324393, -0.5097924 , -0.14171189, -0.12

In [22]:
document_embeddings=np.zeros((func_size, vocab_size))
for idx, doc in enumerate(list_function_keywords):
    doc_vec_sum=np.zeros(vocab_size)
    for term in doc:
        doc_vec_sum+=trained_ft_vectors[term]
    document_embeddings[idx]=doc_vec_sum

In [23]:
document_embeddings[0]

array([-0.18619151,  0.02663726,  0.22246872,  0.12649921, -0.25658372,
       -0.00355002, -0.00182177,  0.10537918, -0.15842131,  0.49702632,
       -0.36332792,  0.19608176, -0.20651202, -0.4605684 ,  0.18327148,
        0.12344187, -0.05747749, -0.05083062,  0.02842423,  0.33967972,
       -0.34382111,  0.17593892,  0.2551797 ,  0.14737436,  0.25548154,
       -0.49370563,  0.2173951 ,  0.28929698, -0.29467228, -0.07052051,
        0.32280469, -0.01454997, -0.18132432, -0.2252001 ,  0.17650607,
        0.12583975, -0.16517092, -0.30670303,  0.02684145,  0.51344556,
        0.1483967 ,  0.07525218, -0.03315615,  0.20330644,  0.28281152,
        0.17892697, -0.25983104,  0.10372335, -0.15665558,  0.14093855,
        0.13444443,  0.06847657, -0.05558093, -0.11429371, -0.07975214,
        0.00077216,  0.1697606 , -0.23654017, -0.07159945,  0.08484866,
        0.02237461, -0.07330912, -0.1006633 , -0.43486297,  0.32682931,
       -0.05835328, -0.01995619, -0.3069028 , -0.08444694, -0.07

In [24]:
print("{} documents with {} dimentions".format(document_embeddings.shape[0], document_embeddings.shape[1]))

1038 documents with 100 dimentions


## Evaluate Model

# Notes

# Questions
The paper mentions 2 evaluation approach: 1 uses Github only, the other one uses both GitHub and StackOverflow. I'm guessing the former one is for tuning in the development stage; while the later is the final evaluation for the completed system (NCS).